# Natural Gas Furnace Module

Models residential natural gas furnace for space heating. Demonstrates the large exergy destruction from using high-grade fuel for low-temperature heating.

## Energy Analysis

**Fuel energy input:**
$$\dot{E}_{fuel,in} = \dot{m}_{fuel} \cdot LHV_{fuel}$$

**Heat output:**
$$\dot{Q}_{heating} = \eta_{AFUE} \cdot \dot{E}_{fuel,in}$$

**Energy efficiency:**
$$\eta_{en,furnace} = \eta_{AFUE}$$

Modern condensing furnace: $\eta_{AFUE} \approx 0.95$ (95%)

## Exergy Analysis

**Fuel exergy input:**
$$\dot{Ex}_{fuel,in} = \dot{m}_{fuel} \cdot e_{ch,fuel} \approx \dot{m}_{fuel} \cdot (\psi_{fuel} \cdot LHV_{fuel})$$

where $\psi_{fuel} = 1.06$ for natural gas.

**Thermal exergy output (Carnot factor):**
$$\dot{Ex}_{heat,out} = \dot{Q}_{heating} \left(1 - \frac{T_0}{T_{heating}}\right)$$

**Exergy efficiency:**
$$\Psi_{furnace} = \frac{\dot{Ex}_{heat,out}}{\dot{Ex}_{fuel,in}} = \eta_{AFUE} \cdot \frac{1}{\psi_{fuel}} \cdot \left(1 - \frac{T_0}{T_{heating}}\right)$$

**Exergy destruction:**
$$\dot{Ex}_{d,furnace} = \dot{Ex}_{fuel,in} - \dot{Ex}_{heat,out}$$

**Key insight:** For typical values ($T_0 = 273K$, $T_{heating} = 323K$), $\Psi_{furnace} \approx 5-7\%$ despite $\eta_{AFUE} = 95\%$!

## Parameters

In [1]:
# Natural Gas Furnace Default Parameters
const FURNACE_DEFAULTS = (
    LHV_ng = 50.0e6,        # Lower heating value of natural gas [J/kg]
    ψ_ng = 1.06,            # Chemical exergy factor for natural gas [-]
    η_AFUE = 0.95,          # Annual Fuel Utilization Efficiency [-]
    T_heating = 323.15,     # Heating delivery temperature [K] (50°C)
    T_0 = 273.15            # Reference/ambient temperature [K] (0°C winter)
)

(LHV_ng = 5.0e7, ψ_ng = 1.06, η_AFUE = 0.95, T_heating = 323.15, T_0 = 273.15)

## Core Functions

In [2]:
"""
    GasFurnace

Struct holding natural gas furnace model parameters.
"""
struct GasFurnace
    Q_rated::Float64        # Rated heating capacity [W]
    η_AFUE::Float64         # Annual Fuel Utilization Efficiency [-]
    LHV::Float64            # Lower heating value of fuel [J/kg]
    ψ_fuel::Float64         # Chemical exergy factor [-]
    T_heating::Float64      # Heating delivery temperature [K]
    
    function GasFurnace(Q_rated::Float64;
            η_AFUE::Float64 = 0.95,
            LHV::Float64 = 50.0e6,
            ψ_fuel::Float64 = 1.06,
            T_heating::Float64 = 323.15)
        new(Q_rated, η_AFUE, LHV, ψ_fuel, T_heating)
    end
end

# Constructor with capacity in kW
function GasFurnace(; Q_kW::Float64, kwargs...)
    GasFurnace(Q_kW * 1000.0; kwargs...)
end

GasFurnace

In [3]:
"""
    carnot_factor(T_hot::Float64, T_0::Float64) -> Float64

Calculate Carnot factor for thermal exergy.
"""
function carnot_factor(T_hot::Float64, T_0::Float64)
    return 1.0 - T_0 / T_hot
end

carnot_factor

In [4]:
"""
    fuel_energy_rate(furnace::GasFurnace, Q_heating::Float64) -> Float64

Calculate required fuel energy input rate [W] for given heat output [W].
"""
function fuel_energy_rate(furnace::GasFurnace, Q_heating::Float64)
    return Q_heating / furnace.η_AFUE
end

fuel_energy_rate

In [5]:
"""
    fuel_mass_rate(furnace::GasFurnace, Q_heating::Float64) -> Float64

Calculate required fuel mass flow rate [kg/s] for given heat output [W].
"""
function fuel_mass_rate(furnace::GasFurnace, Q_heating::Float64)
    E_fuel = fuel_energy_rate(furnace, Q_heating)
    return E_fuel / furnace.LHV
end

fuel_mass_rate

In [6]:
"""
    fuel_exergy_rate(furnace::GasFurnace, Q_heating::Float64) -> Float64

Calculate fuel exergy input rate [W] for given heat output [W].
"""
function fuel_exergy_rate(furnace::GasFurnace, Q_heating::Float64)
    m_fuel = fuel_mass_rate(furnace, Q_heating)
    return m_fuel * furnace.ψ_fuel * furnace.LHV
end

fuel_exergy_rate

In [7]:
"""
    heat_exergy_rate(furnace::GasFurnace, Q_heating::Float64, T_0::Float64) -> Float64

Calculate thermal exergy rate [W] of delivered heat.
"""
function heat_exergy_rate(furnace::GasFurnace, Q_heating::Float64, T_0::Float64)
    η_carnot = carnot_factor(furnace.T_heating, T_0)
    return Q_heating * η_carnot
end

heat_exergy_rate

In [8]:
"""
    exergy_efficiency(furnace::GasFurnace, T_0::Float64) -> Float64

Calculate exergy efficiency of the furnace.
"""
function exergy_efficiency(furnace::GasFurnace, T_0::Float64)
    η_carnot = carnot_factor(furnace.T_heating, T_0)
    return furnace.η_AFUE * η_carnot / furnace.ψ_fuel
end

exergy_efficiency

In [9]:
"""
    analyze_furnace(furnace::GasFurnace, Q_heating::Float64, T_0::Float64) -> NamedTuple

Complete energy and exergy analysis.
Q_heating: heat demand [W], T_0: ambient temperature [K]
"""
function analyze_furnace(furnace::GasFurnace, Q_heating::Float64, T_0::Float64)
    # Validate
    @assert Q_heating >= 0 "Heat demand must be non-negative"
    @assert Q_heating <= furnace.Q_rated "Heat demand exceeds rated capacity"
    @assert T_0 < furnace.T_heating "Ambient must be below heating temp"
    
    # Energy analysis
    E_fuel = fuel_energy_rate(furnace, Q_heating)
    m_fuel = fuel_mass_rate(furnace, Q_heating)
    
    # Exergy analysis
    Ex_fuel = fuel_exergy_rate(furnace, Q_heating)
    Ex_heat = heat_exergy_rate(furnace, Q_heating, T_0)
    Ex_d = Ex_fuel - Ex_heat
    Ψ = exergy_efficiency(furnace, T_0)
    η_carnot = carnot_factor(furnace.T_heating, T_0)
    
    return (
        Q_heating = Q_heating,    # Heat output [W]
        E_fuel = E_fuel,          # Fuel energy input [W]
        m_fuel = m_fuel,          # Fuel mass flow [kg/s]
        Ex_fuel = Ex_fuel,        # Fuel exergy input [W]
        Ex_heat = Ex_heat,        # Heat exergy output [W]
        Ex_d = Ex_d,              # Exergy destruction [W]
        η_en = furnace.η_AFUE,    # Energy efficiency [-]
        η_carnot = η_carnot,      # Carnot factor [-]
        Ψ = Ψ,                    # Exergy efficiency [-]
        T_0 = T_0,                # Ambient temperature [K]
        T_heating = furnace.T_heating  # Heating temperature [K]
    )
end

analyze_furnace

## Tests

In [10]:
using Test

function run_furnace_tests()
    println("Running Gas Furnace Tests...")
    
    # Create test furnace: 20 kW rated
    furnace = GasFurnace(Q_kW=20.0, η_AFUE=0.95, T_heating=323.15)
    T_0 = 273.15  # 0°C ambient
    
    # Test 1: Parameters
    @testset "Parameters" begin
        @test furnace.Q_rated == 20000.0
        @test furnace.η_AFUE == 0.95
        @test furnace.ψ_fuel == 1.06
    end
    
    # Test 2: Carnot factor
    @testset "Carnot Factor" begin
        η_c = carnot_factor(323.15, 273.15)
        @test η_c ≈ 1 - 273.15/323.15 atol=1e-6
        @test 0 < η_c < 1
    end
    
    # Test 3: Energy balance
    @testset "Energy Balance" begin
        Q = 10000.0  # 10 kW
        E_fuel = fuel_energy_rate(furnace, Q)
        @test E_fuel ≈ Q / furnace.η_AFUE atol=1e-6
        @test E_fuel > Q  # Fuel input > heat output
    end
    
    # Test 4: Mass flow
    @testset "Mass Flow" begin
        Q = 10000.0
        m = fuel_mass_rate(furnace, Q)
        E_fuel = fuel_energy_rate(furnace, Q)
        @test m ≈ E_fuel / furnace.LHV atol=1e-10
    end
    
    # Test 5: Exergy balance
    @testset "Exergy Balance" begin
        Q = 10000.0
        Ex_fuel = fuel_exergy_rate(furnace, Q)
        Ex_heat = heat_exergy_rate(furnace, Q, T_0)
        # Ex_fuel = Ex_heat + Ex_d
        @test Ex_fuel > Ex_heat
        Ex_d = Ex_fuel - Ex_heat
        @test Ex_d > 0
    end
    
    # Test 6: Low exergy efficiency
    @testset "Exergy Efficiency" begin
        Ψ = exergy_efficiency(furnace, T_0)
        # Should be very low (5-15%) despite high AFUE
        @test Ψ < 0.15
        @test Ψ > 0.0
        # Much lower than energy efficiency
        @test Ψ < furnace.η_AFUE
    end
    
    # Test 7: Full analysis
    @testset "Full Analysis" begin
        result = analyze_furnace(furnace, 10000.0, T_0)
        # Exergy balance
        @test result.Ex_fuel ≈ result.Ex_heat + result.Ex_d atol=1e-6
        # Efficiency relationship
        @test result.Ψ ≈ result.η_en * result.η_carnot / furnace.ψ_fuel atol=1e-6
    end
    
    # Test 8: Temperature effect on exergy efficiency
    @testset "Temperature Effect" begin
        # Colder ambient = higher Carnot factor = higher exergy efficiency
        Ψ_cold = exergy_efficiency(furnace, 263.15)  # -10°C
        Ψ_warm = exergy_efficiency(furnace, 283.15)  # +10°C
        @test Ψ_cold > Ψ_warm
    end
    
    println("All tests passed!")
end

run_furnace_tests()

Running Gas Furnace Tests...
Test Summary: | Pass  Total  Time
Parameters    |    3      3  0.0s
Test Summary: | Pass  Total  Time
Carnot Factor |    2      2  0.0s
Test Summary:  | Pass  Total  Time
Energy Balance |    2      2  0.0s
Test Summary: | Pass  Total  Time
Mass Flow     |    1      1  0.0s
Test Summary:  | Pass  Total  Time
Exergy Balance |    2      2  0.0s
Test Summary:     | Pass  Total  Time
Exergy Efficiency |    3      3  0.0s
Test Summary: | Pass  Total  Time
Full Analysis |    2      2  0.0s
Test Summary:      | Pass  Total  Time
Temperature Effect |    1      1  0.0s
All tests passed!


## Example Usage

In [11]:
# Example: 20 kW furnace, 10 kW demand, 0°C ambient
furnace = GasFurnace(Q_kW=20.0, η_AFUE=0.95)
result = analyze_furnace(furnace, 10000.0, 273.15)

println("Gas Furnace Analysis")
println("="^45)
println("Rated capacity:     $(furnace.Q_rated/1000) kW")
println("Heat demand:        $(result.Q_heating/1000) kW")
println("Ambient temp:       $(round(result.T_0 - 273.15, digits=1)) °C")
println("Heating temp:       $(round(result.T_heating - 273.15, digits=1)) °C")
println("─"^45)
println("Fuel energy input:  $(round(result.E_fuel/1000, digits=2)) kW")
println("Fuel mass rate:     $(round(result.m_fuel*3600, digits=3)) kg/h")
println("Energy efficiency:  $(round(result.η_en*100, digits=1))%")
println("─"^45)
println("Fuel exergy input:  $(round(result.Ex_fuel/1000, digits=2)) kW")
println("Heat exergy output: $(round(result.Ex_heat/1000, digits=2)) kW")
println("Exergy destruction: $(round(result.Ex_d/1000, digits=2)) kW")
println("Carnot factor:      $(round(result.η_carnot*100, digits=1))%")
println("Exergy efficiency:  $(round(result.Ψ*100, digits=1))%")
println("─"^45)
println("⚠ Despite 95% energy efficiency, only $(round(result.Ψ*100, digits=1))% exergy efficiency!")

Gas Furnace Analysis
Rated capacity:     20.0 kW
Heat demand:        10.0 kW
Ambient temp:       0.0 °C
Heating temp:       50.0 °C
─────────────────────────────────────────────
Fuel energy input:  10.53 kW
Fuel mass rate:     0.758 kg/h
Energy efficiency:  95.0%
─────────────────────────────────────────────
Fuel exergy input:  11.16 kW
Heat exergy output: 1.55 kW
Exergy destruction: 9.61 kW
Carnot factor:      15.5%
Exergy efficiency:  13.9%
─────────────────────────────────────────────
⚠ Despite 95% energy efficiency, only 13.9% exergy efficiency!


In [ ]:
# Compare exergy efficiency at different ambient temperatures (wrapped in let block for NBInclude compatibility)
let
    println("\nExergy Efficiency vs Ambient Temperature")
    println("="^45)
    println("T_amb [°C]   η_carnot [%]   Ψ_furnace [%]")
    println("─"^45)

    for T_amb_C in [-20, -10, 0, 10, 20]
        T_0 = T_amb_C + 273.15
        if T_0 < furnace.T_heating
            η_c = carnot_factor(furnace.T_heating, T_0)
            Ψ = exergy_efficiency(furnace, T_0)
            println("$(lpad(T_amb_C, 8))      $(lpad(round(η_c*100, digits=1), 8))        $(lpad(round(Ψ*100, digits=1), 8))")
        end
    end
end

## Export

Functions available for import:
- `GasFurnace` - Model struct (accepts Q_rated or Q_kW)
- `carnot_factor` - Carnot efficiency for thermal exergy
- `fuel_energy_rate` - Fuel energy input for given heat
- `fuel_mass_rate` - Fuel mass flow rate
- `fuel_exergy_rate` - Fuel exergy input
- `heat_exergy_rate` - Thermal exergy of delivered heat
- `exergy_efficiency` - Furnace exergy efficiency
- `analyze_furnace` - Complete analysis